# Loot hunting in some upredictable environment

There are so many different file formats, which in fact can contain some text. If you want your search engine to be powerful, you should accept various file formats for indexing purposes (remember, *information retrieval* definition states *unstructured* nature of the data).

Let's do some loot hunting and prove you can cope with various formats! Format of the loot is the same as in previous task:

```
LOOT:something\n
```

What you should know about the files:
- Loot is hidden in 7 documents given you by URL. Extensions of these files are `txt, jpg, pdf, docx, html, wav`. Content of loot is **English letters only**, no numbers, spaces or special characters. If you see `0`, this is most probably `O` :)
- In all documents, **except audiofiles**, format is preserved as shown with (maybe) space padding in the end.
- Audiofile just contains a single word. Recognize it and print out.
- There are no dynamic HTML pages, static only.

Input file gives you a direct link to the document. *Print loot values* to the output file.

## Codetest server opportunuties

- Codetest has `requests` library to download the data.
- Codetest has `textract` [library](https://textract.readthedocs.io/en/stable/). It will definitely work (somehow) for pdf and images. It can (and will) fail for speech and docx. But!
  - Codetest has `docx2python` [library](https://pypi.org/project/docx2python/).
  - Codetest has `speech_recognition` [installed](https://pypi.org/project/SpeechRecognition/). This snippet will work in your code:

```
import speech_recognition as sr
r = sr.Recognizer()
with sr.AudioFile('x.wav') as s:
    r.recognize_sphinx(r.record(s))
```

- ... but it will give a wrong result for test :). Please, think how you can overcome this. Maybe use a service API?


<!-- - FireFox is installed, driver is downloaded, `selenium` is (must be) working with no additional tuning. I **BEG** you, please destroy instances of browsers.
```
from selenium import webdriver
x = webdriver.Firefox()
...
x.quit()  # <---- like this
``` -->

1 - http://sprotasov.ru/files/AIR/2/0.txt
2 - http://sprotasov.ru/files/AIR/2/1.docx
3 - http://sprotasov.ru/files/AIR/2/2.pdf
4 - http://sprotasov.ru/files/AIR/2/3.jpg
5 - http://sprotasov.ru/files/AIR/2/4.html
6 - http://sprotasov.ru/files/AIR/2/5a.html
7 - http://sprotasov.ru/files/AIR/2/6.wav

In [1]:
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return "".join(t.strip() for t in visible_texts)

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def get_loot(text):
    m = re.search('L[0|O]{2}T:([a-zA-Z ]*)', text)
    if m:
        return m.group(0)[5:].strip('\n ')
    else:
        return None

def extract_txt(data):
    return get_loot(data.content.decode('utf-8'))

def extract_jpg_pdf(data, extension):
    filename = '{}.{}'.format(extension, extension)
    with open(filename, 'wb+') as f:
        f.write(data.content)
        
    text = textract.process(filename)
    return get_loot(text.decode('utf-8'))

def extract_docx(data):
    filename = '{}.{}'.format('docx', 'docx')
    with open(filename, 'wb+') as f:
        f.write(data.content)
    
    text = d2p(filename).text
    return get_loot(text)

def extract_html(data):
    text = text_from_html(data.content)
    return get_loot(text)

def extract_wav(data):
    filename = '{}.{}'.format('x', 'wav')
    with open(filename, 'wb') as f:
        f.write(data.content)
    r = sr.Recognizer()
    with sr.AudioFile('x.wav') as s:
        text = r.recognize_ibm(r.record(s))
    return text

In [2]:
import requests
import textract
# import docx2python import docx2python as d2p
import re
from bs4 import BeautifulSoup, SoupStrainer
from bs4.element import Comment
import speech_recognition as sr

# with open('input.txt', 'r') as fin:
#     url = fin.read()

url = 'http://sprotasov.ru/files/AIR/2/2.pdf'

data = requests.get(url)
extension = url.split('.')[-1]

if extension == 'txt':
    loot = extract_txt(data)
if extension in ['jpg', 'pdf']:
    loot = extract_jpg_pdf(data, extension)
if extension == 'docx':
    loot = extract_docx(data)
if extension == 'html':
    loot = extract_html(data)
if extension == 'wav':
    loot = extract_wav(data)

print(loot)


FileNotFoundError: [WinError 2] The system cannot find the file specified